# The Task

As per the problem statement, I am required to return the closest questions as per a user's query. Thus in order to understand and learn from the data, I need to gather Questions and Answers that were posted on Stack Overflow. Thus what I need are the following: 

- Title 
- Question body
- Answers for that question
- Votes for each answers

Due to the sheer abundance of data on Stack Overflow and better sanity checks, I restricted the data to only "Python" related questions. However, the entire process in reproducible for other topics as well

# Data Collection

**Google BigQuery** dataset includes an archive of Stack Overflow content, including posts, votes, tags, and badges. This dataset is updated to mirror the Stack Overflow content on the Internet Archive, and is also available through the Stack Exchange Data Explorer. More info about the dataset is given at: https://www.kaggle.com/stackoverflow/stackoverflow

- **bq_helper** is a helper class to perform read-only BigQuery Tasks
- There are many tables on the Stackoverflow database, but we only need concern ourselves with **posts_questions** and **posts_answers**

The query the retrive the required data is *SELECT q.id, q.title, q.body, q.tags, a.body as answers, a.score FROM 'bigquery-public-data.stackoverflow.posts_questions' AS q INNER JOIN 'bigquery-public-data.stackoverflow.posts_answers' AS a ON q.id = a.parent_id WHERE q.tags LIKE '%python%' LIMIT 500000*

1. **SELECT q.id, q.title, q.body, q.tags, a.body as answers, a.score**: Retreives the required columns for the data set
2. **'bigquery-public-data.stackoverflow.posts_questions' AS q INNER JOIN 'bigquery-public-data.stackoverflow.posts_answers' AS a ON q.id = a.parent_id**: This joins the two tables based on the Question ID. Question ID is stored as **id** in "posts_questions" and as **parent_id** in "posts_answers"
3. **WHERE q.tags LIKE '%python%'**: To restrict data to only python related questions
4. **LIMIT 500000**: In order to collect not more than 500,000 data points for faster processing 

In [1]:

import bq_helper

from bq_helper import BigQueryHelper
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="credentials.json"
bq_assistant = BigQueryHelper("bigquery-public-data", "stackoverflow")

QUERY = "SELECT q.id, q.title, q.body, q.tags, a.body as answers, a.score FROM `bigquery-public-data.stackoverflow.posts_questions` AS q INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a ON q.id = a.parent_id WHERE q.tags LIKE '%python%' LIMIT 500000"
df = bq_assistant.query_to_pandas(QUERY)
df.to_csv('data/Original_data.csv')

DefaultCredentialsError: File credentials.json was not found.

# Database Analysis and Pre-processing

## Load Data

In [3]:
import pandas as pd
import numpy as np
import spacy
EN = spacy.load('en_core_web_sm')
df = pd.read_csv('C:/Users/sai ganesh reddy/Desktop/Original_data.csv')
df.head()

ModuleNotFoundError: No module named 'spacy'

In [2]:
print('Datebase shape:' + str(df.shape))

NameError: name 'df' is not defined

## Missing Values

No missing values detected in any of the columns

In [3]:
df.isna().sum()

id         0
title      0
body       0
tags       0
answers    0
score      0
dtype: int64

## Database Manipulation

In order to construct a corpus, I grouped all the answers by concatenating them based on their common questions and tags. Moreover, I added the scores for each answer in order to get a collective score for an entire question

In [99]:
# concat answers for each post
aggregations = {
    'answers':{
        'combined_answers': lambda x: "\n".join(x)
    },
    'score':{
        'combined_score': 'sum'
    }
}
grouped = df.groupby(['id','title', 'body','tags'],as_index=False).agg(aggregations)
deduped_df = pd.DataFrame(grouped)

C:\Users\Rohit\Anaconda3\envs\deeplearning\lib\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [100]:
deduped_df.head()

,id,title,body,tags,answers,score
,,,,,combined_answers,combined_score
0,469,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,python|macos|fonts|photoshop,<p>open up a terminal (Applications->Utilities...,39
1,594,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...,python|sql|database|oracle|cx-oracle,<p>The canonical way is to use the built-in cu...,68
2,742,Class views in Django,"<p><a href=""http://www.djangoproject.com/"" rel...",python|django|views|oop,"<p>I needed to use class based views, but I wa...",75
3,766,Python and MySQL,<p>I can get Python to work with Postgresql bu...,python|mysql|postgresql|bpgsql,<p>I uploaded it and got an internal error</p>...,48
4,1476,How do you express binary literals in Python?,<p>How do you express an integer as a binary n...,python|syntax|binary|integer|literals,<pre><code>&gt;&gt;&gt; print int('01010101111...,352


The following code block shows the result of combining answers and their scores

In [101]:
print('Max score before: ') 
print(np.max(df.score.values))

print('Max score after: ') 
print(np.max(deduped_df.score.values))

Max score before: 
5440
Max score after: 
9730


A couple of helper functions for Text Preprocessing. The steps followed to process a piece of raw text are:

1. Convert raw text into tokens
2. Convert tokens to lower case
3. Remove punctuations
4. Remove Stopwords 

Note: I skipped removal of numeric data since I felt it would remove precious contextual information. I also skipped a 'Stemming/Lemmatization' step because I did not want alter the domain specific terms used in our corpus and risk losing precious information

In [102]:
import re
import nltk
import inflect
from nltk.corpus import stopwords

def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)

def preprocess_text(text):
    return ' '.join(normalize(tokenize_text(text)))

**The raw text for Questions and Answers is given along with the HTML markup with which it was displayed on StackOverflow originally**. 
These refer usually to *p tags, h1-h6 tags and the code tags*

- I constructed a new feature column called 'post_corpus' by combining the title, question body, and all the answers
- I prepended the title to the question body 
- I skipped the 'code' sections because they do not offer useful information for our task
- I constructed urls for each question by appending 'https://stackoverflow.com/questions/' with the question id
- I constructed 2 features for sentiment using the open Source **Textblob library** 

In [103]:
from bs4 import BeautifulSoup
from textblob import TextBlob

title_list = [] 
content_list = []
url_list = []
comment_list = []
sentiment_polarity_list = []
sentiment_subjectivity_list = []
vote_list =[]
tag_list = []
corpus_list = []

for i, row in deduped_df.iterrows():
    title_list.append(row.title.values[0])    # Get question title
    tag_list.append(row.tags.values[0])     # Get question tags
    
    # Questions
    content = row.body.values[0]
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()     # Remove the code section
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
        
    content_list.append(str(row.title.values[0]) + ' ' + str(text))   # Append title and question body data to the updated question body
    
    url_list.append('https://stackoverflow.com/questions/' + str(row.id.values[0]))
    
    # Answers
    content = row.answers.values[0]
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
    comment_list.append(text)
    
    vote_list.append(row.score.values[0])       # Append votes
    
    corpus_list.append(content_list[-1] + ' ' + comment_list[-1])     # Combine the updated body and answers to make the corpus
    
    sentiment = TextBlob(row.answers.values[0]).sentiment
    sentiment_polarity_list.append(sentiment.polarity)
    sentiment_subjectivity_list.append(sentiment.subjectivity)

content_token_df = pd.DataFrame({'original_title': title_list, 'post_corpus': corpus_list, 'question_content': content_list, 'question_url': url_list, 'tags': tag_list, 'overall_scores':vote_list,'answers_content': comment_list, 'sentiment_polarity': sentiment_polarity_list, 'sentiment_subjectivity':sentiment_subjectivity_list})

In [105]:
content_token_df.head()

,original_title,post_corpus,question_content,question_url,tags,overall_scores,answers_content,sentiment_polarity,sentiment_subjectivity
0,How can I find the full path to a font from it...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,https://stackoverflow.com/questions/469,python|macos|fonts|photoshop,39,open up a terminal (Applications->Utilities->T...,0.116667,0.554167
1,cx_Oracle: How do I iterate over a result set?,cx_Oracle: How do I iterate over a result set?...,cx_Oracle: How do I iterate over a result set?...,https://stackoverflow.com/questions/594,python|sql|database|oracle|cx-oracle,68,The canonical way is to use the built-in curso...,0.010082,0.423400
2,Class views in Django,Class views in Django Django view points to a ...,Class views in Django Django view points to a ...,https://stackoverflow.com/questions/742,python|django|views|oop,75,"I needed to use class based views, but I wante...",0.080339,0.439853
3,Python and MySQL,Python and MySQL I can get Python to work with...,Python and MySQL I can get Python to work with...,https://stackoverflow.com/questions/766,python|mysql|postgresql|bpgsql,48,I uploaded it and got an internal error,0.187500,0.390675
4,How do you express binary literals in Python?,How do you express binary literals in Python? ...,How do you express binary literals in Python? ...,https://stackoverflow.com/questions/1476,python|syntax|binary|integer|literals,352,Another way.,-0.064497,0.605969


## Filter for only the most common tags

Each post has a variable number of different tags. In order to narrow down the vast choices for a more accurate model

In [106]:
content_token_df.tags = content_token_df.tags.apply(lambda x: x.split('|'))   # Convert raw text data of tags into lists

# Make a dictionary to count the frequencies for all tags
tag_freq_dict = {}
for tags in content_token_df.tags:
    for tag in tags:
        if tag not in tag_freq_dict:
            tag_freq_dict[tag] = 0
        else:
            tag_freq_dict[tag] += 1

As a personal choice, I decided to with **20 most common tags**, which are shown below

In [107]:
import heapq
most_common_tags = heapq.nlargest(20, tag_freq_dict, key=tag_freq_dict.get)

In [108]:
most_common_tags

['python',
 'python-3.x',
 'django',
 'pandas',
 'python-2.7',
 'numpy',
 'list',
 'matplotlib',
 'dictionary',
 'regex',
 'dataframe',
 'tkinter',
 'string',
 'csv',
 'flask',
 'arrays',
 'tensorflow',
 'json',
 'beautifulsoup',
 'selenium']

The plan is to filter only the data which contains at least one of most_common_tags 

In [109]:
final_indices = []
for i,tags in enumerate(content_token_df.tags.values.tolist()):
    if len(set(tags).intersection(set(most_common_tags)))>1:   # The minimum length for common tags should be 2 because 'python' is a common tag for all
        final_indices.append(i)

In [110]:
final_data = content_token_df.iloc[final_indices]

## Data Normalization

- I created a separate column for the 'processed_title' because I wanted to preserve the original title because I wanted to serve the original titles in the app
- I also normalized the numeric 'scores'

In [111]:
import spacy
EN = spacy.load('en_core_web_sm')

# Preprocess text for 'question_body', 'post_corpus' and a new column 'processed_title'
final_data.question_content = final_data.question_content.apply(lambda x: preprocess_text(x))
final_data.post_corpus = final_data.post_corpus.apply(lambda x: preprocess_text(x))
final_data['processed_title'] = final_data.original_title.apply(lambda x: preprocess_text(x))

# Normalize numeric data for the scores
final_data.overall_scores = (final_data.overall_scores - final_data.overall_scores.mean()) / (final_data.overall_scores.max() - final_data.overall_scores.min())


KeyboardInterrupt



In [ ]:
final_data.tags = final_data.tags.apply(lambda x: '|'.join(x))    # Combine the lists back into text data
final_data.drop(['answers_content'], axis=1)     # Remove the answers_content columns because it is alreaady included in the corpus

In [ ]:
# Save the data
final_data.to_csv('data/Preprocessed_data.csv', index=False)